### Imports 

In [1]:
import spacy
import copy
from scispacy.abbreviation import AbbreviationDetector
import os
from spacy import displacy
from spacy.pipeline import Tagger
import json 
import csv
from collections import defaultdict

### Paths to pipeline,models and data

In [126]:
path_bionlpg13cg = '/home/william/Courses/EDAN70/models/en_ner_bionlp13cg_md-0.2.3/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.3'
path_craft = '/home/william/Courses/EDAN70/models/en_ner_craft_md-0.2.3/en_ner_craft_md/en_ner_craft_md-0.2.3'
path_bc5cdr = '/home/william/Courses/EDAN70/models/en_ner_bc5cdr_md-0.2.4/en_ner_bc5cdr_md/en_ner_bc5cdr_md-0.2.4'
path_jnlpba = '/home/william/Courses/EDAN70/models/en_ner_jnlpba_md-0.2.4/en_ner_jnlpba_md/en_ner_jnlpba_md-0.2.4'

'''path_csv = '/home/william/Courses/EDAN70/data/gold_standard/metadata_gold_standard_subset_10.csv'
path_data = '/home/william/Courses/EDAN70/data/gold_standard/articles/'
path_out = '/home/william/Courses/EDAN70/evaluation/gold_standard/test/'
path_true = '/home/william/Courses/EDAN70/evaluation/gold_standard/true/'''

path_csv = '/home/william/Courses/EDAN70/data/comm_use_subset_100_new/metadata_comm_use_subset_100.csv'
path_data = '/home/william/Courses/EDAN70/data/comm_use_subset_100_new/comm_use_subset_100/'
path_out = '/home/william/Courses/EDAN70/out/'

### Important, Line up model with path and specify which entities to tag

In [127]:
paths_models = [path_bionlpg13cg, path_craft, path_bc5cdr, path_jnlpba]
models = ['bionlpg13cg', 'craft', 'bc5cdr', 'jnlpba']
classes = {'bionlpg13cg': ['GENE_OR_GENE_PRODUCT', 'ORGANISM', 'SIMPLE_CHEMICAL', 'CANCER'], 
           'craft': ['GGP', 'TAXON', 'CHEBI'], 
           'bc5cdr': ['DISEASE', 'CHEMICAL'], 
           'jnlpba': ['PROTEIN']}
classes_set = ['SPECIES', 'DISEASE', 'PROTEIN', 'CHEMICAL']
classes_dict = {'bionlpg13cg': ['SPECIES', 'PROTEIN', 'CHEMICAL', 'DISEASE'], 
           'craft': ['PROTEIN', 'SPECIES', 'CHEMICAL'], 
           'bc5cdr': ['DISEASE', 'CHEMICAL'], 
           'jnlpba': ['PROTEIN'],
           'unified': ['SPECIES', 'DISEASE', 'PROTEIN', 'CHEMICAL'],
           'unified_unique': ['SPECIES', 'DISEASE', 'PROTEIN', 'CHEMICAL']}
sections = ['title', 'abstract', 'texts']
labels = {'ORGANISM': 'SPECIES', 'TAXON': 'SPECIES',
          'CANCER': 'DISEASE', 'DISEASE': 'DISEASE', 
          'GENE_OR_GENE_PRODUCT': 'PROTEIN', 'GGP': 'PROTEIN', 'PROTEIN': 'PROTEIN',
          'SIMPLE_CHEMICAL': 'CHEMICAL', 'CHEBI': 'CHEMICAL', 'CHEMICAL': 'CHEMICAL'} 
priorities = {'bionlpg13cg': {'GENE_OR_GENE_PRODUCT': 59, 'ORGANISM': 12, 'SIMPLE_CHEMICAL': 1, 'CANCER': 1}, 
              'craft': {'GGP': 64, 'TAXON': 18, 'CHEBI': 1}, 
              'bc5cdr': {'DISEASE': 33, 'CHEMICAL': 1}, 
              'jnlpba': {'PROTEIN': 56}}
'''priorities = {'bionlpg13cg': {'GENE_OR_GENE_PRODUCT': 1, 'ORGANISM': 1, 'SIMPLE_CHEMICAL': 1, 'CANCER': 1}, 
              'craft': {'GGP': 1, 'TAXON': 1, 'CHEBI': 1}, 
              'bc5cdr': {'DISEASE': 1, 'CHEMICAL': 1}, 
              'jnlpba': {'PROTEIN': 1}}'''
classes_translation = {'Virus_SARS-CoV' : 'SPECIES', 'Virus_SARS-CoV-2': 'SPECIES', 'Virus_family': 'SPECIES', 
                       'Virus_other': 'SPECIES', 'Disease_COVID-19': 'DISEASE', 'Disease_COVID': 'DISEASE',
                       'Disease_other': 'DISEASE', 'Protein': 'PROTEIN'}

### Load NER models

In [129]:
nlp_models = {}
for i in range(len(paths_models)):
    nlp_models[models[i]] = spacy.load(paths_models[i])

### Scan all file names (not necessarily ordered in the same way as in file folder) and put them in an initial data structure

In [130]:
all_files = os.listdir(path_data)
json_files = [pos_json for pos_json in os.listdir(path_data) if pos_json.endswith('.json')]
data = {}
for filename in json_files:
    with open(path_data + filename) as f:
        data[filename] = json.load(f)

### Add all texts in a dictionary with their cord_uid as key and separate title, abstract, and body texts

In [131]:
keys = data.keys()
structured_data = {}
for key in keys:
    texts = []
    abstract = []
    for i in range(len(data[key]['body_text'])): 
        texts.append(data[key]['body_text'][i]['text'])
    for i in range(len(data[key]['abstract'])):
        abstract.append(data[key]['abstract'][i]['text'])
    structured_data[key] = {'title':data[key]['metadata']['title'], 'abstract':abstract, 'texts':texts}

abstracts = {}
for key in structured_data:
    abstracts[key] = ""
    for i in range(len(structured_data[key]['abstract'])):
        abstracts[key] = abstracts[key] + structured_data[key]['abstract'][i]

### Load CSV data 

In [132]:
csv_data = {}
with open(path_csv, 'r') as csv_file:
    reader = csv.reader(csv_file)
    line_count = 0
    for row in reader:
        if line_count == 0:
            line_count += 1
        else:
            key = row[1]
            keys_helper = key.split("; ")
            for i in range(len(keys_helper)):
                key_temp = keys_helper[i] + '.json'
                for k in keys: 
                    if(key_temp == k):
                        key = key_temp
                        break
      
            csv_data[key] = {'cord_uid': row[0], 'sourcedb': row[2], 'sourceid': row[5], 'title': row[10]}

### Process all articles for each model

In [133]:
doc_models = {}
for key in keys: 
    doc_models[key] = {}
    for model in models:
        abstract = []
        texts = []
        title = nlp_models[model]((structured_data[key]['title']))
            
        for i in range(len(structured_data[key]['abstract'])):
            abstract.append(nlp_models[model]((structured_data[key]['abstract'][i])))
                
        for i in range(len(structured_data[key]['texts'])):
            texts.append(nlp_models[model]((structured_data[key]['texts'][i])))
            
        doc_models[key][model] = {'title':title, 'abstract':abstract, 'texts':texts}

### Tag all entities of the selected classes that are declared at the start

In [134]:
def getSection(id):
    return 'text_' + str(id)

In [135]:
global entities 
entities = dict()

In [137]:
def allEntities():
    entities = dict()
    for k in keys:
        entities[k] = dict()
        for m in models: 
            entities[k][m] = dict()
            for s in sections:
                if(s == 'texts'):
                    for i in range(len(doc_models[k][m][s])):
                        section_temp = getSection(i)
                        entities[k][m][section_temp] = dict()
                else:
                    entities[k][m][s] = dict()

                for c in classes[m]:
                    ents = []

                    if(s == 'title'):
                        for e in doc_models[k][m][s].ents:
                            if(e.label_ == c):
                                ents.append(e)
                        entities[k][m][s][c] = ents

                    elif(s == 'abstract'):
                        ents_abstract = []
                        for i in range(len(doc_models[k][m][s])):
                            for e in doc_models[k][m][s][i].ents:
                                if(e.label_ == c):
                                    ents.append(e)
                        entities[k][m][s][c] = ents

                    elif(s == 'texts'):
                        for i in range(len(doc_models[k][m][s])):
                            section = getSection(i)
                            for e in doc_models[k][m][s][i].ents:
                                if(e.label_ == c):
                                    ents.append(e)
                            entities[k][m][section][c] = ents
                            ents = []
    return entities

In [138]:
entities = allEntities()

### Help methods used to extract the unique entities

Calculates the span 

In [139]:
def calculateSpan(start, end):
    span = end - start
    return span 

Returns true if the first entity has a span >= to that of the second entity

In [140]:
def checkLonger(e1, e2):
    if(calculateSpan(e1.start_char, e1.end_char) >= calculateSpan(e2.start_char, e2.end_char)):
        return True
    return False

Returns true if the two entities are of the same class

In [141]:
def checkType(e1,e2):
    if(labels[e1.label_] == labels[e2.label_]):
        return True
    return False

Returns true if the two entities have at least one character in the same space in common

In [142]:
def checkSame(e1, e2):   
    if(e1.start_char <= e2.end_char and e1.end_char >= e2.start_char):
        return True
    return False

Returns true if the first entity has priority >= to that of the second entity

In [143]:
def checkPriority(m1, m2, c1, c2):
    if(priorities[m1][c1] >= priorities[m2][c2]):
        return True
    return False

In [144]:
def checkIdentical(e1,e2,m1,m2):
    if(m1 == m2 and e1 == e2):
        return True
    return False

In [145]:
def printE(e):
    print(e.text + " - " + str(e.start_char) + " - " + str(e.end_char) + " - " + e.label_)

### Extracting unique entities into a separate list

In [146]:
def runUniqueScript(k, s):
    for m1 in models:
        for c1 in classes[m1]:
            n = -1
            for e1 in entities[k][m1][s][c1]: 
                n = n + 1
                flag = False
                r = {}
    
                for m2 in models: 
                    if(flag):
                        break
                    for c2 in classes[m2]:  
                        m = - 1
                        if(flag):
                            break
                        for e2 in entities[k][m2][s][c2]: 
                            if(flag or checkIdentical(e1,e2,m1,m2)):
                                break 
                    
                            if(checkSame(e1,e2)):
                                if(checkType(e1,e2)):
                                    if(checkLonger(e1,e2)):
                                        r[e2] = [k, m2, s, c2]
                                    else:
                                        entities[k][m1][s][c1].remove(e1)
                                        flag = True
                                else:
                                    if(checkPriority(m1,m2,c1,c2)):
                                        r[e2] = [k, m2, s, c2]
                                    else:
                                        entities[k][m1][s][c1].remove(e1)
                                        flag = True
                                   
                if(not flag):
                    for e in r.keys():
                        entities[r[e][0]][r[e][1]][r[e][2]][r[e][3]].remove(e)
                                 

In [147]:
def uniqueEntities():
    for k in keys:
        for s in sections:
            if(s == 'texts'):
                for i in range(len(structured_data[k]['texts'])):
                    section = getSection(i)
                    runUniqueScript(k,section)
            else:
                runUniqueScript(k,s)

### FILE WRITER, Uses Pubannotation format

In [156]:
def addDenotation(string, e, model, label, id):
    string = string + "\n    {"
    string = string + "\n      \"id\": \"" + model + "_T" + str(id) + "\","
    #string = string + "\n      \"id\": \"" + label + "\","
    string = string + "\n      \"span\": {"
    string = string + "\n        \"begin\": " + str(e.start_char) + "," 
    string = string + "\n        \"end\": " + str(e.end_char)
    string = string + "\n      },"
    #string = string + "\n      \"obj\": \"" + model + "_T" + str(id) + "\""
    string = string + "\n      \"obj\": \"" + label + "\""
    string = string + "\n    },"
    return string

In [149]:
def addDenotations(string, key, model, section):
    id = 1
    bool = False
    string = string + "\n  \"denotations\": ["
        
    for m in model: 
        for c in classes[m]:
            for e in entities[key][m][section][c]:
                string = addDenotation(string, e, m, labels[c], id)
                bool = True
                id = id + 1
   
    if(bool):
        string = string[:-1]
        string = string + "\n  ] \n}"
    else:
        string = string + "]\n}"
    return string

In [150]:
def writer(key, id):
    #string = "{ \n  \"cord_uid\": \"" + key + "\"," --- When CSV is missing
    string = "{ \n  \"cord_uid\": \"" + csv_data[key]['cord_uid'] + "\","
    string = string + "\n  \"sourcedb\": \"" + csv_data[key]['sourcedb'] + "\","
    string = string + "\n  \"sourceid\": \"" + csv_data[key]['sourceid'] + "\","
    string + "\n  \"divid\": \"" + str(id) + "\","
    return string 

In [151]:
def writerTitle(key, model, section):
    string = "\n".ljust(4) + "\"text\": \"" + structured_data[key][section].replace('"', '\\"') + "\","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, model, section)
    return string

In [152]:
def writerAbstract(key, model, section):
    string = "\n".ljust(4) + "\"text\": \"" + abstracts[key].replace('"', '\\"') + "\","
    #string = "\n    \"text\": " + abstracts[key] + ","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, model, section)
    return string

In [153]:
def writerBody(key, model, section, n):
    string = "\n".ljust(4) + "\"text\": \"" + structured_data[key]['texts'][n].replace('"', '\\"') + "\","
    string = string + "\n  \"project\": \"cdlai_CORD-19\","
    string = addDenotations(string, key, model, section)
    return string

In [154]:
def out(path, model):
    for key in keys:
        name_title = csv_data[key]['cord_uid'] + '-0-title.json'
        #name_title = key + '-0-title.json' --- When CSV is missing
        with open(path + name_title, 'w') as file:
            string = writer(key, 0) + writerTitle(key, model, sections[0])
            file.write(string)
        
        if(len(structured_data[key]['abstract']) != 0):
            name_abstract = csv_data[key]['cord_uid'] + '-1-abstract.json'
            #name_abstract = key + '-1-abstract.json' --- When CSV is missing
            with open(path + name_abstract, 'w') as file:
                string = writer(key, 1) + writerAbstract(key, model, sections[1])
                file.write(string)

        for i in range(len(structured_data[key]['texts'])):
            n = i + 2
            name_text = csv_data[key]['cord_uid'] + '-' + str(n) + '-body_text.json'
            #name_text = key + '-' + str(n) + '-body_text.json' --- When CSV is missing
            section = getSection(i)
            with open(path + name_text, 'w') as file:
                string = writer(key, n) + writerBody(key, model, section, i)
                file.write(string)

In [157]:
entities = allEntities()
for model in models: 
    path = path_out + model + '/'
    #print(path)
    out(path, [model])
    
path_unified = path_out + 'unified/'
out(path_unified, models)

uniqueEntities()
path_unified_unique = path_out + 'unified_unique/'
out(path_unified_unique, models)
entities = allEntities()

In [99]:
for e in entities:
    print(e)

PMC7003341.json
PMC7054940.json
PMC6988272.json
PMC7077245.json
PMC7110798.json
PMC7033720.json
PMC7094943.json
31996494.json
32013309.json
PMC7159299.json


### Evaluator

In [100]:
true_files = os.listdir(path_data)
json_true_files = [pos_json for pos_json in os.listdir(path_true) if pos_json.endswith('.json')]
d = {}
for filename in json_true_files:
    with open(path_true + filename) as f:
        d[filename] = json.load(f)

for k in d.keys(): 
    for e in d[k]['denotations']:
        if((e['id']) in classes_translation):
            e['id'] = classes_translation[e['id']]

for k in d.keys():
    with open(path_true + k, 'w') as file:
        json.dump(d[k], file, indent=4)


### Evaluator from Dict. group: Annie and Sofie, thanks! 

In [101]:
"""
PubAnnotation evaluator for COVID-19

Authors:
    Annie Tallind, Lund University, Faculty of Engineering
    Kaggle ID: atllnd
    Github ID: annietllnd

    Sofi Flink, Lund University, Faculty of Engineering
    Kaggle ID: sofiflinck
    Github ID: obakanue

    TODO:
     - Should we check correct word class?
"""
import os
import json
# true_positives = some_function() # number of true positives
# false_positives = some_other_function() # number of false positives
# true_negatives = other_function() # number of true negatives
# false_negatives = last_one() # number of false negatives
# for any is_checked = False in tagger_output_dict -> False positives
# for any is_checked = False in true_output_dict -> False negatives


def print_progress(nbr_pubannotations_evaluated, total_pubannotations):
    """
    Prints estimated progress based on number of total articles and number of articles processed.
    """
    print_progress_bar(nbr_pubannotations_evaluated,
                       total_pubannotations,
                       prefix='EVALUATION PROGRESS\t',
                       suffix='COMPLETE')


def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█'):
    """
    Author: StackOverflow
            User Greenstick
            Question 30740258
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end='', flush=True)
    # Print New Line on Complete
    if iteration == total:
        print()


class PubannotationEvaluator:
    """
    PubAnnotationEvaluator evaluates output compared to a true output, the arguments are the directory paths to the
    location of the outputs and a set containing what word classes/dictionaries to be evaluated.
    """
    def __init__(self, tagger_output_dir_path, true_output_dir_path, word_classes_set):
        self.tagger_output_dicts = dict()
        self.true_output_dicts = dict()
        self.word_classes_result_dict = dict()
        self.recall_values = list()
        self.precision_values = list()

        self.word_classes_set = word_classes_set
        self.true_positives = self.true_negatives = self.false_positives = self.false_negatives = \
            self.nbr_true_entities = self.recall_value = self.precision_value = self.total_true_positives = \
            self.total_false_positives = self.total_false_negatives = self.iteration_nbr = 0

        self.__generate_result_dict(self.word_classes_set)

        self.__load_output(tagger_output_dir_path, 1)
        self.__load_output(true_output_dir_path, 0)

        self.processes_total = len(self.tagger_output_dicts) + len(self.word_classes_result_dict)

    def __generate_result_dict(self, word_classes_set):
        """
        Initializes a dictionary containing all the results for respective word class.
        """
        for word_class in word_classes_set:
            self.word_classes_result_dict[word_class] = {'total': {'amount': 0,
                                                                   'entities': list()
                                                                   },
                                                         'true_positives': {'amount': 0,
                                                                            'entities': list()
                                                                            },
                                                         'false_positives': {'amount': 0,
                                                                             'entities': list()
                                                                             },
                                                         'false_negatives': {'amount': 0,
                                                                             'entities': list()
                                                                             }
                                                         }

    def __load_output(self, dir_output_path, is_tagger_output):
        """
        Loads output files from a given directory in to corresponding dictionary. Second argument indicates if it is
        the true output or the output to be evaluated.
        """
        output_paths = os.listdir(dir_output_path)
        for pubannotation_file_name in output_paths:
            if pubannotation_file_name == '.DS_Store':  # For MacOS users skip .DS_Store-file
                continue                                # generated.
            full_path = dir_output_path + pubannotation_file_name
            with open(full_path) as pubannotation_obj:
                pubannotation_dict = json.loads(pubannotation_obj.read())
                pubannotation_dict.update({'is_checked': False})
                if is_tagger_output:
                    self.tagger_output_dicts.update({pubannotation_file_name: pubannotation_dict})
                else:
                    self.true_output_dicts.update({pubannotation_file_name: pubannotation_dict})

    def evaluate(self):
        """
        Evaluates outputs compared to true outputs.
        """
        self.__compare_outputs()
        self.__evaluate_word_class()
        self.__calculate_micro()
        self.__print_result('MICRO')
        self.__calculate_macro()
        self.__print_result('MACRO')
        self.__calculate_harmonic_mean()

    def __compare_outputs(self):
        """
        Iterates through all outputs to be compared to through output and compare output denotations.
        """
        for cord_uid in self.tagger_output_dicts:
            print_progress(self.iteration_nbr, self.processes_total)
            tagger_pubannotation = self.tagger_output_dicts[cord_uid]
            if cord_uid in self.true_output_dicts:
                true_pubannotation = self.true_output_dicts[cord_uid]
                text = true_pubannotation['text']
                word_classes_list = [denotations_list_element['id'] for denotations_list_element in
                                     true_pubannotation['denotations']]
                self.__compare_output(tagger_pubannotation,
                                      true_pubannotation,
                                      cord_uid,
                                      word_classes_list,
                                      text)
                self.iteration_nbr += 1
        print_progress(self.iteration_nbr, self.processes_total)

    def __compare_output(self, tagger_pubannotation, true_pubannotation, cord_uid, word_classes_list, text):
        """
        Compares denotations with true denotations, false negatives field are incremented if there is a an existing
        match in true denotations that does not exist in denotations to be compared, and only for the word classes
        existing in the list argument. When a denotation is checked the field 'is_checked' is set to True which
        helps to calculate false positives and false negatives. If two denotations are matching in span true positives
        filed will be incremented in the result dictionary.
        """
        tagger_denotations = tagger_pubannotation['denotations']
        for tagger_denotation in tagger_denotations:
            tagger_denotation['is_checked'] = False
        true_denotations = true_pubannotation['denotations']
        for true_denotation in true_denotations:
            true_denotation['is_checked'] = False

        for tagger_denotation in tagger_denotations:
            i = 0
            for true_denotation in true_denotations:
                tagger_denotation_span = (tagger_denotation['span']['begin'], tagger_denotation['span']['end'])
                true_denotation_span = (true_denotation['span']['begin'], true_denotation['span']['end'])
                if tagger_denotation_span == true_denotation_span:
                    # Might want to change to a safer implementation where we don't depend on an ordered
                    # word_classes_list. TODO

                    if word_classes_list[i] in self.word_classes_set and tagger_denotation['id'] == true_denotation['id']:
                        word_class = word_classes_list[i]
                        self.word_classes_result_dict[word_class]['true_positives']['amount'] += 1
                        self.word_classes_result_dict[word_class]['total']['amount'] += 1
                        self.word_classes_result_dict[word_class]['true_positives']['entities'].append(
                            f'id: {word_class}, '
                            f'entity: {text[tagger_denotation_span[0]:tagger_denotation_span[1] + 1]}, '
                            f'span: {tagger_denotation_span}')
                        self.word_classes_result_dict[word_class]['total']['entities'].append(
                            f'id: {word_class}, '
                            f'entity: {text[tagger_denotation_span[0]:tagger_denotation_span[1] + 1]}, '
                            f'span: {tagger_denotation_span}')
                        true_denotation.update({'is_checked': True})
                        tagger_denotation.update({'is_checked': True})
                        break
                i += 1

        for tagger_denotation in tagger_denotations:
            if not tagger_denotation['is_checked']:
                word_class = tagger_denotation['id']
                if word_class in self.word_classes_set:
                    tagger_denotation_span = (tagger_denotation['span']['begin'], tagger_denotation['span']['end'])
                    self.word_classes_result_dict[word_class]['false_positives']['amount'] += 1
                    self.word_classes_result_dict[word_class]['false_positives']['entities'].append(
                        f'id: {word_class}, '
                        f'entity: {text[tagger_denotation_span[0]:tagger_denotation_span[1] + 1]}, '
                        f'span: {tagger_denotation_span}')
                tagger_denotation.update({'is_checked': True})
        for true_denotation in true_denotations:
            word_class = true_denotation['id']
            if word_class in self.word_classes_set:
                if not true_denotation['is_checked']:
                    true_denotation_span = (true_denotation['span']['begin'], true_denotation['span']['end'])
                    self.word_classes_result_dict[word_class]['false_negatives']['amount'] += 1
                    self.word_classes_result_dict[word_class]['total']['amount'] += 1
                    self.word_classes_result_dict[word_class]['false_negatives']['entities'].append(
                        f'id: {word_class}, '
                        f'entity: {text[true_denotation_span[0]:true_denotation_span[1] + 1]}, '
                        f'span: {true_denotation_span}')
                true_denotation.update({'is_checked': True})



    def __evaluate_word_class(self):
        """
        Evaluates results for each word class.
        """
        for word_class in self.word_classes_result_dict:
            print_progress(self.iteration_nbr, self.processes_total)
            self.__precision(word_class)
            self.__recall(word_class)
            self.__print_result(word_class)
            self.iteration_nbr += 1
        print_progress(self.iteration_nbr, self.processes_total)

    def __precision(self, word_class):
        """
        Calculates precision figure.
        """
        true_positives = self.word_classes_result_dict[word_class]['true_positives']['amount']
        false_positives = self.word_classes_result_dict[word_class]['false_positives']['amount']
        self.total_true_positives += true_positives
        self.total_false_positives += false_positives
        sum_value = true_positives + false_positives
        if sum_value:
            self.precision_value = true_positives / sum_value
            self.precision_values.append(self.precision_value)
        else:
            print('########### WARNING ###########')
            print(f'{word_class} found no match, the precision result can be misleading')
            print("########### WARNING ###########")
            self.precision_value = 0

    def __recall(self, word_class):
        """
        Calculates recall figure.
        """
        true_positives = self.word_classes_result_dict[word_class]['true_positives']['amount']
        false_negatives = self.word_classes_result_dict[word_class]['false_negatives']['amount']
        self.total_false_negatives += false_negatives
        sum_value = true_positives + false_negatives
        if sum_value:
            self.recall_value = true_positives / sum_value
            self.recall_values.append(self.recall_value)
        else: 
            print('########### WARNING ###########')
            print(f"'{word_class}' found no match, the recall result can be misleading")
            print('########### WARNING ###########')
            print('\n')
            self.precision_values.append(0)
            self.recall_value = 0

    def __calculate_micro(self):
        """
        Calculates micro figure.
        """
        sum_value = self.total_true_positives + self.total_false_positives
        if sum_value:
            self.precision_value = self.total_true_positives / (self.total_true_positives + self.total_false_positives)
            self.recall_value = self.total_true_positives / (self.total_true_positives + self.total_false_negatives)
        else:
            self.precision_value = 0
            self.recall_value = 0

    def __calculate_macro(self):
        """
        Calculates macro figure.
        """
        self.precision_value = 0
        self.recall_value = 0
        for precision_value in self.precision_values:
            self.precision_value += precision_value
        for recall_value in self.recall_values:
            self.recall_value += recall_value
        if self.precision_value:
            self.precision_value /= len(self.precision_values)
            self.recall_value /= len(self.recall_values)

    def __calculate_harmonic_mean(self):
        """
        Calculates harmonic mean/F1 score figure.
        """
        sum_value = self.precision_value + self.recall_value
        harmonic_mean = 0
        if sum_value:
            harmonic_mean = (2*self.precision_value*self.recall_value) / sum_value
        print(f'#########\tHARMONIC MEAN RESULT:\t###########')
        print(f'Harmonic mean:\t{harmonic_mean * 100:.0f}%')

    def __print_result(self, word_class):
        """
        Prints result for a given section/word class.
        """
        print(f'\n\n#########\t{word_class.upper()} PRECISION & RECALL RESULT:\t###########')
        print('\n')
        print(f'Precision:\t{self.precision_value * 100:.0f}%')
        print(f'Recall:\t\t{self.recall_value * 100:.0f}%')
        print('\n')

    def get_total_entities(self, word_class):
        """
        Returns a list of total entities for a word class.
        """
        return self.word_classes_result_dict[word_class]['total']['entities']

    def get_true_positive_entities(self, word_class):
        """
        Returns a list of entities marked as true positives for a word class.
        """
        return self.word_classes_result_dict[word_class]['true_positives']['entities']

    def get_false_positive_entities(self, word_class):
        """
        Returns a list of entities marked as false positives for a word class.
        """
        return self.word_classes_result_dict[word_class]['false_positives']['entities']

    def get_false_negative_entities(self, word_class):
        """
        Returns a list of entities marked as false negatives for a word class.
        """
        return self.word_classes_result_dict[word_class]['false_negatives']['entities']

    def get_total(self, word_class):
        """
        Returns number of entities associated with a word class.
        """
        return self.word_classes_result_dict[word_class]['total']['amount']

    def get_true_positives(self, word_class):
        """
        Returns number of true positives associated with a word class.
        """
        return self.word_classes_result_dict[word_class]['true_positives']['amount']

    def get_false_positives(self, word_class):
        """
        Returns number of false positives associated with a word class.
        """
        return self.word_classes_result_dict[word_class]['false_positives']['amount']

    def get_false_negatives(self, word_class):
        """
        Returns number of false negatives associated with a word class.
        """
        return self.word_classes_result_dict[word_class]['false_negatives']['amount']

In [125]:
model = 'unified_unique'
tagger_output_dir_path = path_out + model + '/'
true_output_dir_path = path_true
evaluator = PubannotationEvaluator(tagger_output_dir_path, true_output_dir_path, ['DISEASE', 'SPECIES', 'PROTEIN']) #classes_dict[model])
evaluator.evaluate()


'''model = 'bionlpg13cg'
tagger_output_dir_path = path_out + model + '/'
true_output_dir_path = path_true
evaluator = PubannotationEvaluator(tagger_output_dir_path, true_output_dir_path, classes_set)
evaluator.evaluate()'''


'''print("-------------- \n\nVALUES -----------------------" )
print('False positives: ' + str(evaluator.get_false_positives('SPECIES')) + '\n')
print('False negatives: ' + str(evaluator.get_false_negatives('SPECIES')) + '\n')
print('True positives: ' + str(evaluator.get_true_positives('SPECIES')) + '\n')
print('Total: ' + str(evaluator.get_total('SPECIES')) + '\n')
print('False positive entities: ' + str(evaluator.get_false_positive_entities('SPECIES')) + '\n')
print('False negative entities: ' + str(evaluator.get_false_negative_entities('SPECIES')) + '\n')
print('True positive entities: ' + str(evaluator.get_true_positive_entities('SPECIES')) + '\n')
print('Total entities: ' + str(evaluator.get_total_entities('SPECIES')) + '\n')'''

#print(evaluator.get_true_positives('SPECIES'))
#for i in range(len(evaluator.get_total_entities('SPECIES'))):
  #  print(evaluator.get_total_entities('SPECIES')[i])


EVALUATION PROGRESS	 |██████████████████████████████████████████████████████████████████████████████████████--------------| 87.0% COMPLETE

#########	DISEASE PRECISION & RECALL RESULT:	###########


Precision:	24%
Recall:		50%


EVALUATION PROGRESS	 |███████████████████████████████████████████████████████████████████████████████████████████---------| 91.3% COMPLETE

#########	SPECIES PRECISION & RECALL RESULT:	###########


Precision:	13%
Recall:		16%


EVALUATION PROGRESS	 |███████████████████████████████████████████████████████████████████████████████████████████████-----| 95.7% COMPLETE

#########	PROTEIN PRECISION & RECALL RESULT:	###########


Precision:	29%
Recall:		91%


EVALUATION PROGRESS	 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% COMPLETE


#########	MICRO PRECISION & RECALL RESULT:	###########


Precision:	20%
Recall:		33%




#########	MACRO PRECISION & RECALL RESULT:	###########


Precision:	22%
Recall:		5

'print("-------------- \n\nVALUES -----------------------" )\nprint(\'False positives: \' + str(evaluator.get_false_positives(\'SPECIES\')) + \'\n\')\nprint(\'False negatives: \' + str(evaluator.get_false_negatives(\'SPECIES\')) + \'\n\')\nprint(\'True positives: \' + str(evaluator.get_true_positives(\'SPECIES\')) + \'\n\')\nprint(\'Total: \' + str(evaluator.get_total(\'SPECIES\')) + \'\n\')\nprint(\'False positive entities: \' + str(evaluator.get_false_positive_entities(\'SPECIES\')) + \'\n\')\nprint(\'False negative entities: \' + str(evaluator.get_false_negative_entities(\'SPECIES\')) + \'\n\')\nprint(\'True positive entities: \' + str(evaluator.get_true_positive_entities(\'SPECIES\')) + \'\n\')\nprint(\'Total entities: \' + str(evaluator.get_total_entities(\'SPECIES\')) + \'\n\')'